# Expression remapper

This notebook is a simple app that allows users to map different facial expressions and motions to keyboard inputs. I aim to use this to play a video game like Dark Souls as a test.

Using MediaPipe, and following their [face landmark detection guide](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python).

## Install dependencies

#### Mediapipe
Using for computer vision.

In [ ]:
!pip install mediapipe
# opencv as well

### Pynput
Allows us to emulate keyboard inputs

In [1]:
!pip install pynput

     ---------------------------------------- 0.0/89.2 kB ? eta -:--:--
     ---------------------------------------- 89.2/89.2 kB 5.3 MB/s eta 0:00:00


## Import dependencies

In [1]:
# Mediapipe for face landmark detection
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# OpenCV for drawing utilities and webcam input
import cv2

# Generally useful for math
import math
import time

# Pynput for simulating keyboard inputs
from pynput.keyboard import Key, Controller

## Load the model

In [2]:
model_path = 'models/face_landmarker.task'

## Utility functions for extracting head orientation

In [3]:
#!! import math

# Toggles whether the remapping is currently active. If paused, no keypresses will be simulated.
remapping_paused = True

# State variables
keyboard = Controller()
moving_up = False
moving_down = False
moving_left = False
moving_right = False
turning_left = False
turning_right = False

# Mouth states
jaw_state = "closed"
jaw_open_thresh = 0.1

# Mouth pucker states
mouth_puckered = False

# Mouth shrug state
mouth_shrugging = False
mouth_shrug_thresh = 0.1

# Squint states
L_squinting = False
R_squinting = False
squint_thresh = 0.3

# Eyebrow states
L_brow_raised = False

# Real yaw pitch and roll
yaw = 0.0 # Rotate left/right
pitch = 0.0 # Tilt up/down
roll = 0.0 # Tilt left/right

# Calibrated yaw pitch and roll values
calib_yaw = 0
calib_pitch = 0
calib_roll = 0

# The rest positions, used to calculate the calibrated values
rest_yaw = 0.0
rest_pitch = 0.0
rest_roll = 0.0


# Extract the yaw, pitch, and roll of the head from the facial transformation matrix
# (yaw, pitch, and roll are easier for me to work with than a matrix)
def update_yaw_pitch_roll(mtrix):
    # Get access to the global variables
    global yaw
    global pitch
    global roll

    # Calculate the yaw pitch and roll from the transformation matrix (conversion)
    if mtrix[0][0] == 1.0:
        yaw = math.atan2(mtrix[0][2], mtrix[2][3])
        pitch = 0
        roll = 0

    elif mtrix[0][0] == -1.0:
        yaw = math.atan2(mtrix[0][0], mtrix[2][3])
        pitch = 0
        roll = 0

    else:
        yaw = math.atan2(-mtrix[2][0], mtrix[0][0])
        pitch = math.atan2(-mtrix[1][2], mtrix[1][1])
        roll = math.asin(mtrix[1][0])

    # Calculate the CALIBRATED yaw pitch roll values
    global calib_yaw
    global calib_pitch
    global calib_roll
    calib_yaw = yaw - rest_yaw
    calib_pitch = pitch - rest_pitch
    calib_roll = roll - rest_roll

    # Simulate key presses based on the head's orientation
    if not remapping_paused:
        global moving_left
        global moving_right
        global moving_up
        global moving_down
        
        #print(math.degrees(calib_pitch))
        thresh = 10.0
        # Look Left / Right
        if math.degrees(calib_yaw) > thresh: # Head is facing left
            keyboard.release('l')
            moving_right = False
            if not moving_left:
                keyboard.press('j')
                moving_left = True
        elif math.degrees(calib_yaw) < -thresh: # Head is facing right
            keyboard.release('j')
            moving_left = False
            if not moving_right:
                keyboard.press('l')
                moving_right = True
        else: # Head is in neutral position
            keyboard.release('j')
            moving_left = False
            keyboard.release('l')
            moving_right = False
        
        
        # Tilt Left / Right
        if math.degrees(calib_roll) > thresh: # Head is tilted right
            keyboard.release('a')
            turning_left = False
            if not moving_right:
                keyboard.press('d')
                turning_right = True
        elif math.degrees(calib_roll) < -thresh: # Head is tilted left
            keyboard.release('d')
            turning_right = False
            if not moving_left:
                keyboard.press('a')
                turning_left = True
        else: # Head is in neutral position
            keyboard.release('a')
            turning_left = False
            keyboard.release('d')
            turning_right = False
        
        
        # Look Up / Down
        if math.degrees(calib_pitch) > thresh:
            keyboard.release('s')
            moving_up = False
            if not moving_down:
                keyboard.press('w')
                moving_down = True
        elif math.degrees(calib_pitch) < -thresh:
            keyboard.release('w')
            moving_down = False
            if not moving_up:
                keyboard.press('s')
                moving_up = True
        else: # Head is in neutral position
            keyboard.release('w')
            moving_down = False
            keyboard.release('s')
            moving_up = False

def check_blendshapes():
    print("blendshapes")
    
def calibrate_yaw_pitch_roll():
    # Get access to global rest pos variables (I don't like python..)
    # then set them to the current yaw, pitch, and roll of the user's head.
    global yaw
    global pitch
    global roll
    global rest_yaw
    global rest_pitch
    global rest_roll
    rest_yaw = yaw
    rest_pitch = pitch
    rest_roll = roll
    
    # Print results
    print("resting yaw is: " + str(rest_yaw))
    print("resting pitch is: " + str(rest_pitch))
    print("resting roll is: " + str(rest_roll))
    

## Main program

In [5]:
#!! import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode
    

# Create a face landmarker instance with the live stream mode:
def print_result(result: FaceLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    
    # If any landmarks have been detected..
    if result.face_landmarks:
        
        # Get the matrix from face landmarker
        transf_matrix = result.facial_transformation_matrixes[0]
        update_yaw_pitch_roll(transf_matrix)   
        
        # Check the state of different face blendshapes
        bshapes = result.face_blendshapes[0]

        # Jaw Open
        global jaw_state
        if bshapes[25].score > jaw_open_thresh:
            if not jaw_state == "open":
                keyboard.press('7')
                jaw_state = "open"
                #print("jaw open")
        else: # Jaw closed
            if not jaw_state == "closed":
                keyboard.release('7')
                jaw_state = "closed"
                #print("jaw closed")
                
        # Mouth pucker
        global mouth_puckered
        if bshapes[38].score > 0.8:
            if not mouth_puckered:
                keyboard.press(Key.space)
                time.sleep(0.1)
                keyboard.release(Key.space)
                mouth_puckered = True
        else:
            if mouth_puckered:
                mouth_puckered = False
        
        # Brow outer up left
        global L_brow_raised
        if bshapes[4].score > 0.4:
            if not L_brow_raised:
                keyboard.press('q')
                L_brow_raised = True
        else:
            if L_brow_raised:
                L_brow_raised = False
        
#         # Left squint
#         global L_squinting
#         global R_squinting
#         if bshapes[19].score > 0.3 and bshapes[20].score < 0.3:
#             if not L_squinting:
#                 with keyboard.pressed('a'):
#                     time.sleep(0.2)
#                     keyboard.press(Key.space)
#                     time.sleep(0.1)
#                     keyboard.release(Key.space)
#                 #keyboard.release('a')
#                 L_squinting = True
#                 R_squinting = False
#         # Right squint
#         elif bshapes[19].score < 0.3 and bshapes[20].score > 0.3:
#             if not R_squinting:
#                 with keyboard.pressed('d'):
#                     time.sleep(0.2)
#                     keyboard.press(Key.space)
#                     time.sleep(0.1)
#                     keyboard.release(Key.space)
#                 #keyboard.release('d')
#                 R_squinting = True
#                 L_squinting = False
#         else: # If both are open, or both are squinting..
#                 R_squinting = False
#                 L_squinting = False
                
                
                
#         # Mouth shrug lower
         
#         global mouth_shrugging
#         if bshapes[42].score > mouth_shrug_thresh:
#             if not mouth_shrugging:
#                 keyboard.press(Key.space)
#                 mouth_shrugging = True
#                 time.sleep(0.1)
#                 keyboard.release(Key.space)
#         else:
#             if mouth_shrugging:
#                 mouth_shrugging = False

        
        
    
    
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    output_facial_transformation_matrixes=True, ## TRY CHANGING THIS
    output_face_blendshapes=True,
    num_faces=1) ## TRY CHANGING THIS

with FaceLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.
    # Use OpenCV’s VideoCapture to start capturing from the webcam.
    video = cv2.VideoCapture(0)
    
    # Initialise the frame timestamp (MAKE THIS IN ms?)
    frame_timestamp = 0
    prev_frame_time = 0
    new_frame_time = 0
    
    # Print user instructions
    print("---- Calibration -----\n")
    print("Please look straight ahead at your screen, then press ENTER when comfortable\nThis will calibrate your 'neutral' position.")
    print("When you are ready to start playing, press '0' and keyboard inputs will begin to be simulated as you move your head.\n\nTo stop, press '0' again.\n\n")
    print("---- How to use -----\n")
    print("Tilt your head left/right (roll) to simulate pressing 'a' and 'd' respectively\n")
    print("Tilt your head forward/back (pitch) to simulate pressing the 'w' and 's' keys.\n")
    print("You can refer to this image for a reference of what 'pitch' and 'roll' mean: \nhttps://miro.medium.com/v2/resize:fit:604/format:webp/0*3aYfZkNKTeobv07d.png")
    
    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while video.isOpened():
        ret, frame = video.read()
        
        # break if there's no video input
        if not ret: 
            break
        
        # Record the current time this frame
        new_frame_time = time.time()
        
        # Convert the frame received from OpenCV to a MediaPipe Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # To improve performance, apparently
        frame.flags.writeable = False
        
        # Send live image data to perform face landmarking.
        # The results are accessible via the `result_callback` provided in
        # the `PoseLandmarkerOptions` object.
        # The pose landmarker must be created with the live stream mode.
        landmarker.detect_async(mp_image, frame_timestamp)
        
        # To improve performance, apparently
        frame.flags.writeable = True
        
        # Increment the timestamp
        frame_timestamp += 1
        
        # Calculate and display FPS
        fps = 1/(new_frame_time-prev_frame_time) 
        prev_frame_time = new_frame_time 

        fps = int(fps) # convert fps to int (from float) 
        fps = str(fps) # convert fps to str (from int) so it can be displayed

        # drawing FPS counter to the frame
        cv2.putText(frame, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)
    
        # Draw the output
        cv2.imshow('window', frame)
        
        # Listen for keyboard input
        keyPressed = cv2.waitKey(5)
        if keyPressed == ord('q'): # 'q': Break out of loop and exit program
            break
            
        elif keyPressed == 13: # 'Enter': Calibrate yaw+pitch+roll
            calibrate_yaw_pitch_roll()
            
        elif keyPressed == ord('0'): # '0': Toggle the the remapping
            global remapping_paused
            remapping_paused = not remapping_paused
            print("remapping paused set to: {}".format(remapping_paused))
        
        
        #time.sleep(1/1000)

# Clean up
video.release()
cv2.destroyAllWindows()

---- Calibration -----

Please look straight ahead at your screen, then press ENTER when comfortable
This will calibrate your 'neutral' position.
When you are ready to start playing, press '0' and keyboard inputs will begin to be simulated as you move your head.

To stop, press '0' again.


---- How to use -----

Tilt your head left/right (roll) to simulate pressing 'a' and 'd' respectively

Tilt your head forward/back (pitch) to simulate pressing the 'w' and 's' keys.

You can refer to this image for a reference of what 'pitch' and 'roll' mean: 
https://miro.medium.com/v2/resize:fit:604/format:webp/0*3aYfZkNKTeobv07d.png
resting yaw is: 0.038827234834839026
resting pitch is: -0.18704108629607896
resting roll is: 0.04360882554542435
remapping paused set to: False
resting yaw is: -0.0037661564129292213
resting pitch is: -0.25781759310373076
resting roll is: 0.08102221050324661
